In [ ]:
from sudoku_mrv import generate_board, verify_board

In [ ]:
a = generate_board(completeness=20, outer_grid_size=9)

In [ ]:
a

In [ ]:
verify_board(a)

In [ ]:
from train import train
from model import DiscreteDiffusion


outer_grid_size = 9
model = DiscreteDiffusion(outer_grid_size=outer_grid_size)
trained_model = train(model, outer_grid_size=outer_grid_size)